In [1]:
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.tools
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import skimage
import Images
import ModelUNet_rep
import ModelUNet_v2 #version 2 learns areas and boundaries of nuclei
import matplotlib.pyplot as plt

import copy


/home/lienhard/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [31]:
train=Images.Images("../input/stage1_train")

#set aside 10% for validation
val1=train.subset(np.arange(train.n()*.9, train.n()))
train=train.subset(np.arange(train.n()*.9,))

model2=ModelUNet_v2.ModelUNet(name='unet_256x256_xent_v2')
model1=ModelUNet_rep.ModelUNet(name='unet_v1_256x256')



In [32]:
val1.load_images()
val1.load_masks()
val2=copy.deepcopy(val1)

val1.add_predictions(model1)
print("expected LB score of v1 (val): {}".format(np.mean(val1.features['iou_score'])))

val2.add_predictions(model2,boundary_height=0.06)
val2.features.drop(['ids'], axis=1).head()
print("expected LB score(val): {}".format(np.mean(val2.features['iou_score'])))
#somehow its mutch worse than that...

100%|██████████| 67/67 [00:09<00:00,  7.27it/s]


predicting foreground...
labeling predictions...


/home/lienhard/.local/lib/python3.6/site-packages/skimage/segmentation/_join.py:118: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.

  0%|          | 0/67 [00:00<?, ?it/s]

computing scores... 


100%|██████████| 67/67 [00:17<00:00,  3.85it/s]


...

labeling predictions...


  0%|          | 0/67 [00:00<?, ?it/s]

computing scores... 


100%|██████████| 67/67 [00:16<00:00,  4.13it/s]

expected LB score(val): 0.46328323712665526


In [42]:
df1=val1.features.add_suffix("_v1")
df2=val2.features.add_suffix("_v2")

df=pd.concat([df1,df2], axis=1)
df['n']=df.index
df['description']=pd.Series("IOU(v2: " + (df.iou_score_v2).astype(str)+"<br>ID: " + (df.n).astype(str))
df_bw=df[(df.n_channels_v1==1)]
df_col=df[(df.n_channels_v1==3)]
df.head()

,ids_v1,size_x_v1,size_y_v1,n_channels_v1,nuclei_n_v1,nuclei_meanSz_v1,nuclei_stdSz_v1,nuclei_minSz_v1,nuclei_maxSz_v1,n_pred_v1,...,iou_th60_v2,iou_th65_v2,iou_th70_v2,iou_th75_v2,iou_th80_v2,iou_th85_v2,iou_th90_v2,iou_th95_v2,n,description
0,e5f8ad0f0a43af8ca57e31e16800108abdfb44a7e962a7...,256,256,1,6,198,72,85,282,6.0,...,1.000000,1.000000,1.000000,1.000000,0.333333,0.090909,0.000000,0.0,0,IOU(v2: 0.6424242424242423<br>ID: 0
1,e66a97b2c77f3d66a7d3cebbc6a36c8c6259368a397f7b...,256,256,1,11,179,91,21,347,10.0,...,0.833333,0.833333,0.692308,0.692308,0.571429,0.222222,0.047619,0.0,1,IOU(v2: 0.555921855921856<br>ID: 1
2,e66f25e175abab08ecb4e5f6859db64a211e0ddffb262d...,256,256,1,11,144,66,48,259,11.0,...,1.000000,0.833333,0.833333,0.571429,0.222222,0.047619,0.047619,0.0,2,IOU(v2: 0.5555555555555556<br>ID: 2
3,e7a3a7c99483c243742b6cfa74e81cd48f126dcef00401...,520,696,1,150,564,183,40,1071,110.0,...,0.863354,0.863354,0.829268,0.754386,0.612903,0.276596,0.027397,0.0,3,IOU(v2: 0.6012785192936023<br>ID: 3
4,e81c758e1ca177b0942ecad62cf8d321ffc315376135bc...,256,256,1,47,442,340,27,1932,43.0,...,0.468750,0.362319,0.270270,0.205128,0.119048,0.068182,0.032967,0.0,4,IOU(v2: 0.2609459485099387<br>ID: 4


In [36]:
trace_bw=go.Scatter(x=df_bw.nuclei_n_v1, y=df_bw.n_pred_v1, mode='markers', marker=dict(size=12, line=dict(width=1), color="black"),
                    name="grayscale", text=df_bw.description,)
trace_col=go.Scatter(x=df_col.nuclei_n_v1, y=df_col.n_pred_v1, mode='markers', marker=dict(size=12, line=dict(width=1), color="red"),
                    name="color", text=df_col.description,)
data=[trace_bw, trace_col]
layout=go.Layout(title="number of nuclei / image", hovermode="closest",
                xaxis=dict(title="# truth", ticklen=5, zeroline=False, gridwidth=2),
                yaxis=dict(title="# prediction", ticklen=5,  gridwidth=2),)
fig=go.Figure(data=data, layout=layout)
plot(fig, show_link=False)

'file:///project/42/lienhard/github/kaggle/data-science-bowl-2018/working/temp-plot.html'

In [43]:
trace_bw=go.Scatter(x=df_bw.nuclei_n_v2, y=df_bw.iou_score_v2, mode='markers', marker=dict(size=12, line=dict(width=1), color="black"),
                    name="grayscale", text=df_bw.description,)
trace_col=go.Scatter(x=df_col.nuclei_n_v2, y=df_col.iou_score_v2, mode='markers', marker=dict(size=12, line=dict(width=1), color="red"),
                    name="color", text=df_col.description,)
data=[trace_bw, trace_col]
layout=go.Layout(title="number of nuclei / image", hovermode="closest",
                xaxis=dict(title="# truth", ticklen=5, zeroline=False, gridwidth=2),
                yaxis=dict(title="score", ticklen=5,  gridwidth=2),)
fig=go.Figure(data=data, layout=layout)
plot(fig, show_link=False)
#1) the more nuclei the worse
#2) color images are really bad

'file:///project/42/lienhard/github/kaggle/data-science-bowl-2018/working/temp-plot.html'

In [48]:
trace_bw=go.Scatter(x=df_bw.iou_score_v1, y=df_bw.iou_score_v2, mode='markers', marker=dict(size=12, line=dict(width=1), color="black"),
                    name="grayscale", text=df_bw.description,)
trace_col=go.Scatter(x=df_col.iou_score_v1, y=df_col.iou_score_v2, mode='markers', marker=dict(size=12, line=dict(width=1), color="red"),
                    name="color", text=df_col.description,)
data=[trace_bw, trace_col]
layout=go.Layout(title="number of nuclei / image", hovermode="closest",
                xaxis=dict(title="score v1", ticklen=5, zeroline=False, gridwidth=2),
                yaxis=dict(title="score v2", ticklen=5,  gridwidth=2),)
fig=go.Figure(data=data, layout=layout)
plot(fig, show_link=False)


'file:///project/42/lienhard/github/kaggle/data-science-bowl-2018/working/temp-plot.html'